# FastText

<br>

- **Word2Vec의 확장**으로 볼 수 있음.
- 차이점? Word2Vec은 단어를 쪼개질 수 없는 단위로 생각하는 반면, **FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주.**
    - **내부 단어 (subword)를 고려**해 학습.

## 내부 단어(subword)의 학습

<br>

- FastText에선 **각 단어를 글자 단위 n-gram의 구성으로 취급.**
    - n을 몇으로 결정하는지에 따라 단어들이 얼마나 분리되는지 결정됨.
    - ***시작과 끝을 의미하는 <, > 를 도입.*** 예시를 보자.

```{.python}
# apple에 대해 n = 3인 경우, 단어 분리 후 벡터 생성
<ap, app, ppl, ple, le>
```
- 추가적으로 **기존 단어에 <, >를 붙인 토큰을 하나 더 벡터화** 함. **특별 토큰**.
```{.python}
# 특별토큰
<apple>
```
- 즉, n=3인 경우 FastText는 *단어 apple에 대해 5개의 일반 토큰과 1개의 특별 토큰을 벡터화* 한다.
```{.python}
<ap, app, ppl, ple, le>, <apple>
```

<br>

- 실 사용시엔 n의 최소값과 최대값으로 범위를 설정할 수 있음. default는 3,6.
    - 최소값 = 3, 최대값 = 6인 경우, apple에 대해 FastText는 다음과 같은 단어들을 벡터화 한다.
    ```{.python}
    # n = 3 ~ 6 인 경우
    <ap, app, ppl, ppl, le>, <app, appl, pple, ple>, <appl, pple>, ... , <apple>
    ```
    - **내부 단어를 벡터화 한다는 의미는 이 단어들에 대해 Word2Vec을 수행한다**는 의미.
    - 내부 단어들의 벡터값을 얻었다면, apple의 벡터값은 위의 **벡터값들의 총 합**으로 구성.
    ```{.python}
    apple = <ap + app + ppl + ppl + le> + <app + appl + , ..., +<apple> 
    ```
- 이런 방법은 Word2Vec에선 가질 수 없었던 강점을 가지게 됨.

## 모르는 단어(Out Of Vocabulary, OOV)에 대한 대응

<br>

- FastText의 신경망을 학습한 후엔, 데이터 셋의 모든 단어의 각 n-gram에 대해 워드 임베딩이 진행됨.
- Word2Vec(and GloVe)에 비해 생기는 강점이 뭘까?
    - 데이터 셋만 충분하다면, **내부단어를 통해 모르는 단어에 대해서도 다른 단어와의 유사도 계산이 가능.**
    - birthplace라는 단어를 학습하지 않은 상태라고 해보자.
    - 다른 단어에서 ***birth와 place라는 내부 단어***가 있다면,
    - FastText는 **birthplace의 벡터를 얻을 수 있다.**

## 단어 집합 내 빈도수가 적었던 단어(Rare Word)에 대한 대응

<br>

- Word2Vec은 등장 빈도수가 적은 단어에 대해서 임베딩 정확도가 높지 않았었음.
    - **참고할 수 있는 경우의 수가 적다보니 정확하게 임베딩이 되지 않았던 것.**
- FastText는 **단어가 희귀 단어라도, 그 단어의 n-gram이 다른 단어의 n-gram과 겹치는 경우**라면
- Word2Vec과 비교했을 때 **비교적 높은 임베딩 벡터 값을 얻을 수 있음.**

<br>

- 이런 점 때문에 FastText는 **노이즈가 많은 말뭉치에서 강점**을 갖게 됨.
- 실제 많은 비정형 데이터엔 오타나 맞춤법이 틀린 단어가 많이 섞여있음.
    - ***이런 단어들은 당연히 등장 빈도수가 매우 적음.*** 일종의 희귀 단어.
    - Word2Vec에서 이런 단어는 임베딩이 제대로 안됐지만, FastText는 일정 수준의 성능을 보여줌.

## Word2Vec vs. FastText 간단 비교

<br>

- 이전에 썼던 ted xml파일 그대로 사용. 다시 코드 쓰자 ㅠ

In [1]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from lxml import etree
import warnings

warnings.filterwarnings(action='ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Cheol Hee
[nltk_data]     Kim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
targetText = etree.parse(targetXML)

# context block 내용만 추출,
# regex써서 괄호 배경음 부분 제거.
parseText = '\n'.join(targetText.xpath('//content/text()'))
contentText = re.sub(r'\([^)]*\)', '', parseText)

# 입력 corpus에 대해 NLTK를 통해 문장 토큰화 진행 (불용어 제거)
sentText = sent_tokenize(contentText)

# 구두점 제거, 대문자->소문자 변환.
normalizedText = []
for text in sentText:
    tokens = re.sub(r"^a-z0-9", " ", text.lower())
    normalizedText.append(tokens)
    
# 각 문장 토큰화 진행
result = [word_tokenize(sentence) for sentence in normalizedText]

print('총 샘플 수 : {}'.format(len(result)))

총 샘플 수 : 273424


### Word2Vec Model

In [4]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences = result,
    vector_size = 100,
    window = 5,
    min_count = 5,
    workers = 4,
    sg = 0
)

In [5]:
model.wv.most_similar('electrofishing')

KeyError: "Key 'electrofishing' not present"

    단어 집합에 electrofishing이 없다고 함.
    Word2Vec은 학습 데이터에 존재하지 않는 단어(모르는 단어)에 대해선
    임베딩 벡터가 존재하지 않으므로 유사도 계산이 안됨.

### FastText

<br>

- Word2Vec 학습 코드만 FastText로 변경해 실행해보자.

In [6]:
from gensim.models import FastText

model = FastText(
    result,
    vector_size = 100,
    window = 5,
    min_count = 5,
    workers = 4,
    sg = 1
)

In [7]:
model.wv.most_similar('electrofishing')

[('electrolux', 0.8584972023963928),
 ('electrolyte', 0.8558030724525452),
 ('electroshock', 0.8482690453529358),
 ('electroencephalogram', 0.8378998637199402),
 ('airbag', 0.8358634114265442),
 ('electric', 0.8316196799278259),
 ('airbus', 0.8284210562705994),
 ('electrogram', 0.8203827738761902),
 ('electron', 0.8109163641929626),
 ('electrode', 0.8094940185546875)]

    Word2Vec과는 다르게
    학습하지 않은 단어에 대해 유사한 단어를 계산해 출력하고 있음.